In [1]:
import pandas as pd
import re
import tarfile
import gzip
from bs4 import BeautifulSoup

In [2]:
!pwd

/home/jupyter/notebooks/archive.org/commercial


In [3]:
df = pd.read_json('commercial.json')
df

,archive_id,intervals,video_id
0,CNNW_20160727_000000_Anderson_Cooper_360,"[[0, 12245], [1370837, 1401568], [2355588, 257...",1
1,MSNBCW_20161206_070000_The_11th_Hour_With_Bria...,"[[1016983, 1217033], [1500517, 1716399], [1815...",2
2,CNNW_20161229_130000_New_Day,"[[1019352, 1199349], [1766032, 2004237], [2321...",3
3,CNNW_20161231_220000_CNN_Newsroom_With_Poppy_H...,"[[730563, 890540], [1045111, 1202452], [141094...",4
4,MSNBCW_20150220_180000_Ronan_Farrow_Daily,"[[742842, 932432], [1372005, 1518885], [165642...",5
...,...,...,...
304968,FOXNEWSW_20230715_060000_Hannity,"[[1151150, 1314296], [1870919, 1885900], [2450...",362930
304969,MSNBCW_20230714_100000_Morning_Joe,"[[1081980, 1340906], [2228526, 2480744], [3558...",362931
304970,CNNW_20230715_020000_Whos_Talking_to_Chris_Wal...,"[[0, 66866], [678077, 918317], [1439404, 16847...",362932
304971,FOXNEWSW_20230714_150000_The_Faulkner_Focus,"[[2746360, 3040520], [3345875, 3631010]]",362933


In [4]:
# tar tvzf html-2017.tar.gz > html-2017.txt
archive_id = []
with open('../shared/html-2022.txt') as f:
    for l in f:
        a = l.split('/')
        fn = a[-1].strip()
        if fn.endswith('.html'):
            aid = fn.replace('.html', '')
            archive_id.append(aid)

In [5]:
df2022 = pd.DataFrame({'archive_id': archive_id})
df2022

,archive_id
0,TVRAIN_20190717_033000
1,TVRAIN_20171220_010000
2,ALJAZ_20200406_113200_Inside_Story_2020_Ep__96
3,BBCNEWS_20200718_213000_The_Papers
4,CNNW_20190929_090000_CNN_Newsroom_Live
...,...
917251,KGO_20180428_120000_ABC7_News_500AM
917252,CSPAN_20210827_102900_President_Biden_Remarks_...
917253,KGO_20190412_000000_ABC7_News_500PM
917254,BBCNEWS_20200206_030000_BBC_News


In [6]:
cdf = df[df.archive_id.isin(df2022.archive_id)].copy()
cdf

,archive_id,intervals,video_id
39,FOXNEWSW_20171120_160000_Happening_Now,"[[708024, 874657], [1343443, 1504020], [177537...",42
41,MSNBCW_20171117_110000_Morning_Joe,"[[1908041, 2149783], [2449816, 2677293], [3167...",44
43,MSNBCW_20171101_170000_MSNBC_Live_With_Craig_M...,"[[887437, 1105822], [1737320, 1970186], [25781...",46
47,CNNW_20171203_100000_CNN_Newsroom_Live,"[[0, 18885], [973523, 1123556], [1556690, 1746...",50
57,MSNBCW_20171112_150000_AM_Joy,"[[0, 12629], [858808, 1048932], [1615331, 1782...",60
...,...,...,...
287357,FOXNEWSW_20221026_140000_Americas_Newsroom,"[[809492, 1083999], [1621252, 1921252], [23713...",344949
287358,CNNW_20221026_210000_The_Situation_Room_With_W...,"[[725725, 949264], [1488637, 1707889], [249390...",344950
287359,MSNBCW_20221026_040000_Alex_Wagner_Tonight,"[[1290505, 1560625], [2100281, 2142857], [2176...",344951
287360,MSNBCW_20221026_170000_Chris_Jansing_Reports,"[[10744, 58958], [1050649, 1260876], [1788470,...",344952


In [7]:
cdf['html_file'] = 'html/' + cdf['archive_id'] + '.html'
cdf

,archive_id,intervals,video_id,html_file
39,FOXNEWSW_20171120_160000_Happening_Now,"[[708024, 874657], [1343443, 1504020], [177537...",42,html/FOXNEWSW_20171120_160000_Happening_Now.html
41,MSNBCW_20171117_110000_Morning_Joe,"[[1908041, 2149783], [2449816, 2677293], [3167...",44,html/MSNBCW_20171117_110000_Morning_Joe.html
43,MSNBCW_20171101_170000_MSNBC_Live_With_Craig_M...,"[[887437, 1105822], [1737320, 1970186], [25781...",46,html/MSNBCW_20171101_170000_MSNBC_Live_With_Cr...
47,CNNW_20171203_100000_CNN_Newsroom_Live,"[[0, 18885], [973523, 1123556], [1556690, 1746...",50,html/CNNW_20171203_100000_CNN_Newsroom_Live.html
57,MSNBCW_20171112_150000_AM_Joy,"[[0, 12629], [858808, 1048932], [1615331, 1782...",60,html/MSNBCW_20171112_150000_AM_Joy.html
...,...,...,...,...
287357,FOXNEWSW_20221026_140000_Americas_Newsroom,"[[809492, 1083999], [1621252, 1921252], [23713...",344949,html/FOXNEWSW_20221026_140000_Americas_Newsroo...
287358,CNNW_20221026_210000_The_Situation_Room_With_W...,"[[725725, 949264], [1488637, 1707889], [249390...",344950,html/CNNW_20221026_210000_The_Situation_Room_W...
287359,MSNBCW_20221026_040000_Alex_Wagner_Tonight,"[[1290505, 1560625], [2100281, 2142857], [2176...",344951,html/MSNBCW_20221026_040000_Alex_Wagner_Tonigh...
287360,MSNBCW_20221026_170000_Chris_Jansing_Reports,"[[10744, 58958], [1050649, 1260876], [1788470,...",344952,html/MSNBCW_20221026_170000_Chris_Jansing_Repo...


In [8]:
html_files = cdf['html_file'].tolist()
html_files[:20]

['html/FOXNEWSW_20171120_160000_Happening_Now.html',
 'html/MSNBCW_20171117_110000_Morning_Joe.html',
 'html/MSNBCW_20171101_170000_MSNBC_Live_With_Craig_Melvin.html',
 'html/CNNW_20171203_100000_CNN_Newsroom_Live.html',
 'html/MSNBCW_20171112_150000_AM_Joy.html',
 'html/FOXNEWSW_20171107_000000_The_Story_With_Martha_MacCallum.html',
 'html/CNNW_20171113_130000_New_Day.html',
 'html/FOXNEWSW_20171206_170000_Outnumbered.html',
 'html/FOXNEWSW_20171110_060000_Hannity.html',
 'html/FOXNEWSW_20171123_230000_Special_Report_With_Bret_Baier.html',
 'html/CNNW_20171107_200000_CNN_Newsroom_With_Brooke_Baldwin.html',
 'html/CNNW_20171122_080000_CNN_Newsroom_Live.html',
 'html/MSNBCW_20171202_230000_MSNBC_Joy_Reid.html',
 'html/FOXNEWSW_20171204_030000_Fox_News_Sunday_With_Chris_Wallace.html',
 'html/MSNBCW_20171202_130000_Hugh_Hewitt.html',
 'html/CNNW_20171113_200000_CNN_Newsroom_With_Brooke_Baldwin.html',
 'html/MSNBCW_20171117_070000_The_11th_Hour_With_Brian_Williams.html',
 'html/CNNW_201711

In [9]:
def is_in_intervals(intervals, start, end):
    for (a, b) in intervals:
        if (a <= start <= b) or (a <= end <= b):
            return True
    return False

In [10]:
%%time

i = 0
data = []
ofn = 'archive-cc-2022-nc.csv.gz'
first = True
with tarfile.open('../shared/html-2022.tar.gz', 'r:gz') as tar:
    for tar_info in tar:
        if tar_info.isfile():
            if tar_info.name in html_files:
                fname = tar_info.name
                i += 1
                if i % 100 == 0:
                    # output to file every 100 records
                    print(i, fname)
                    tdf = pd.DataFrame(data)
                    data = []
                    if first:
                        tdf.to_csv(ofn, index=False, header=first, compression='gzip')
                        first = False
                    else:
                        tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')
                try:
                    fobj = tar.extractfile(tar_info)
                    html = fobj.read()
                    soup = BeautifulSoup(html, 'html.parser')
                    html = soup.prettify()
                    soup = BeautifulSoup(html, 'html.parser')
                    intervals = cdf[cdf.html_file==tar_info.name]['intervals'].iloc[0]
                    text = ""
                    for a in soup.find_all('div', {'class': 'tvcol'}):
                        l = a.find('a')
                        href = l.attrs['href']
                        m = re.match('/start/(\d+)/end/(\d+)', href)
                        if m:
                            start = int(m.group(1)) * 1000
                            end = int(m.group(2)) * 1000
                            t = a.find('div', {'class': 'snipin nosel'})
                            if not is_in_intervals(intervals, start, end):
                                text += t.text.strip()
                    archive_id = fname.replace('html/', '').replace('.html', '')
                    data.append({'archive_id': archive_id, 'text_nc': text})
                except Exception as e:
                    print(e, fname)

# last chunk
tdf = pd.DataFrame(data)
tdf.to_csv(ofn, mode='a', index=False, header=first, compression='gzip')

100 html/CNNW_20180916_230000_CNN_Newsroom_With_Ana_Cabrera.html
200 html/CNNW_20180702_000000_CNN_Films.html
300 html/FOXNEWSW_20180630_140000_Cavuto_Live.html
400 html/CNNW_20221011_230000_Erin_Burnett_OutFront.html
500 html/CNNW_20191117_170000_State_of_the_Union_With_Jake_Tapper.html
600 html/CNNW_20171211_200000_CNN_Newsroom_With_Brooke_Baldwin.html
700 html/FOXNEWSW_20210121_100000_FOX__Friends_First.html
800 html/FOXNEWSW_20210603_050000_Tucker_Carlson_Tonight.html
900 html/CNNW_20180712_030000_CNN_Tonight_With_Don_Lemon.html
1000 html/CNNW_20200522_130000_CNN_Newsroom_With_Poppy_Harlow_and_Jim_Sciutto.html
1100 html/MSNBCW_20201205_230000_American_Voices_With_Alicia_Menendez.html
1200 html/MSNBCW_20190516_150000_MSNBC_Live_With_Craig_Melvin.html
1300 html/CNNW_20180218_170000_State_of_the_Union_With_Jake_Tapper.html
1400 html/MSNBCW_20180915_100000_Dateline.html
1500 html/CNNW_20171211_100000_Early_Start_with_Christine_Romans_and_Dave_Briggs.html
1600 html/FOXNEWSW_20180214_140